In [1]:
import pandas as pd
import requests
import io

In [2]:
state_info = pd.read_csv('../source_data/state_info.csv', dtype = 'object')

df_spy = pd.read_csv('../source_data/SPY.csv')

df_spy = df_spy[['Date', 'Adj Close']]
df_spy.columns = ['date', 'price']

intervals = [1,3,6,12,48]
for i in intervals:
    df_spy[f'spy_{i}'] = df_spy['price'].pct_change(i)

df_spy = df_spy[df_spy['date'].str.match(r'^\d{4}-11-\d{2}$')]
df_spy['year'] = df_spy['date'].str[:4]

df_spy = df_spy[['year'] + [f'spy_{i}' for i in intervals]]

In [3]:
df_nasdaq = pd.read_csv('../source_data/NASDAQCOM.csv')
df_nasdaq = df_nasdaq.sort_values(by = 'DATE')
df_nasdaq['month'] = df_nasdaq['DATE'].str[0:-3]
df_nasdaq = df_nasdaq.groupby('month')\
            .first()\
            .reset_index()\
            .sort_values(by = 'month')

intervals = [1,3,6,12,48]
for i in intervals:
    df_nasdaq[f'nasdaq_{i}'] = df_nasdaq['NASDAQCOM'].pct_change(i)

df_nasdaq = df_nasdaq[df_nasdaq['month'].str[-2:] == '11']
df_nasdaq['year'] = df_nasdaq['month'].str[:4]
df_nasdaq = df_nasdaq[['year'] + [f'nasdaq_{i}' for i in intervals]]

In [5]:
df = pd.merge(left = df_nasdaq, right = df_spy, on = 'year', how = 'outer')
df.to_csv('../processed_data/1971-2023-stocks.csv', index = False)